# Eksempel 01: Rask Chat via OpenAI SDK

Denne notatboken viser hvordan man bruker OpenAI SDK med Microsoft Foundry Local for raske chat-interaksjoner.

## Oversikt

Dette eksempelet viser:
- Bruk av OpenAI Python SDK med Foundry Local
- Håndtering av både Azure OpenAI- og lokale Foundry-konfigurasjoner
- Implementering av riktig feilhåndtering og fallback-strategier
- Bruk av FoundryLocalManager for tjenestestyring


## Forutsetninger

Sørg for at du har:
1. Foundry Local installert og kjørende
2. En modell lastet inn (f.eks. `foundry model run phi-4-mini`)
3. Nødvendige Python-pakker installert

### Installer avhengigheter


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## Importer nødvendige biblioteker


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## Konfigurasjon

Sett opp konfigurasjonen din. Du kan bruke enten:
1. **Azure OpenAI** (skybasert)
2. **Foundry Local** (lokal inferens)

### Alternativ 1: Azure OpenAI-konfigurasjon


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### Alternativ 2: Foundry Lokal Konfigurasjon


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## Klientinitialisering

Opprett OpenAI-klienten basert på din konfigurasjon:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## Grunnleggende Chat Eksempel

La oss teste en enkel chat-interaksjon:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## Interaktiv chatsesjon

Prøv ulike typer spørsmål for å se hvordan modellen reagerer:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## Avansert bruk: Strømming av svar

For lengre svar kan strømming gi en bedre brukeropplevelse:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## Feilhåndtering og diagnostikk

La oss sjekke tjenestestatusen og tilgjengelige modeller:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## Tilpasset Prompt Testing

Bruk cellen nedenfor for å teste dine egne prompts:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## Sammendrag

Denne notatboken demonstrerte:

1. **✅ Klientoppsett**: Hvordan konfigurere OpenAI SDK med Foundry Local
2. **✅ Grunnleggende Chat**: Enkle forespørsel-svar-interaksjoner
3. **✅ Streaming**: Generering av svar i sanntid
4. **✅ Feilhåndtering**: Robust feilhåndtering og diagnostikk
5. **✅ Tjenestestatus**: Sjekke modelltilgjengelighet og tjenestestatus

### Neste steg

- **Eksempel 02**: Avansert SDK-integrasjon med støtte for Azure OpenAI
- **Eksempel 04**: Bygge en Chainlit-chatapplikasjon
- **Eksempel 05**: Orkestreringssystemer med flere agenter
- **Eksempel 06**: Intelligent modellruting

### Viktige fordeler med Foundry Local

- 🔒 **Personvern**: Data forlater aldri enheten din
- ⚡ **Hastighet**: Lav ventetid ved lokal inferens
- 💰 **Kostnad**: Ingen kostnader for API-bruk
- 🔌 **Offline**: Fungerer uten internettforbindelse
- 🛠️ **Kompatibilitet**: OpenAI-kompatibel API
